# Lecture 1: introduction to `qibo`

### Introduction

During this course we are going to use `Qibo`, an open source framework for quantum computing. It provides us with an high level language which can be used to implement algorithms with both circuit-based and adiabatic computation approaches and, once the code is set up, it can be easily executed on various engines, including both classical and quantum hardware. 

<center><img src="figures/qibo_ecosystem.svg" alt="drawing" width="800"/></center>

For more info about the whole framework one can have a look to the [`qibo` webpage](https://qibo.science/).

### Setup
We start installing `qibo` and `qibolab` and then importing some useful primitives.

In [ ]:
# if you don't have already qibo and qibolab in your computer uncomment and execute the following line

# !pip install qibo
# !pip install qibolab[emulator]

A crucial step is the backend choice. In qibo four backends are provided, and can be used for different kind of applications.

<center><img src="figures/backends.svg" alt="drawing" width="800"/></center>

The blue backends correspond to classical hardware, while the red one can be selected if we want to execute our algorithm directly on a quantum computer.

In [ ]:
# some imports from qibo + numpy
import numpy as np
import qibo
from qibo import Circuit, gates, hamiltonians

In [ ]:
# with the following line we set the desired backend
qibo.set_backend(backend="numpy")

### Build my first `qibo` circuit

Now we are ready to code our first quantum circuit using `qibo`.

In [ ]:
# set the number of qubits
nqubits = 4

# we initialise the circuit using the Circuit class
c = Circuit(nqubits=nqubits)

c.draw()

We can now fill the circuits with some quantum gates. Here are some of the quantum gates which are available in Qibo.
<center><img src="figures/gates.png" alt="drawing" width="400"/></center>
Let's apply a $X$ gate on the first qubit and a $H$ gate on the third qubit:

In [ ]:
c.add(gates.X(0))
c.add(gates.H(2))
c.draw()

Finally we add measurements gate on all qubits.

In [ ]:
c.add(gates.M(i) for i in range(4))
c.draw()

In [ ]:
result = c(nshots=1000)

In [ ]:
print(result) # visualize ket
print(result.state()) # visualize state in computational basis
print(result.probabilities()) # visualize probabilities
print(result.samples()) # visualize samples
print(result.frequencies()) # visualize frequencies

We can now visualize the state.

In [ ]:
import matplotlib.pyplot as plt
def visualize_states(counter):
    """Plot state's frequencies."""
        
    fig, ax = plt.subplots(figsize=(5, 5 * 6/8))
    ax.set_title('State visualization')
    ax.set_xlabel('States')
    ax.set_ylabel('#')
    plt.xticks(rotation=90)
    n = len(list(counter)[0])
    bitstrings = [format(i, f"0{n}b") for i in range(2**n)]
    for state in bitstrings:
        ax.bar(state, counter[state] if state in counter else 0, color='#C194D8', edgecolor="black")
visualize_states(result.frequencies())

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
    <strong>Exercise:</strong> Write a quantum circuit with 3 qubits to produce the states $|001\rangle$, $|010\rangle$, and $|111\rangle$.
</div>

In [ ]:
circuit = Circuit(3)
circuit.add(gates.X(2))
circuit.add(gates.M(i) for i in range(3))
visualize_states(circuit(nshots=1000).frequencies())

In [ ]:
circuit = Circuit(3)
circuit.add(gates.X(1))
circuit.add(gates.M(i) for i in range(3))
visualize_states(circuit(nshots=1000).frequencies())

In [ ]:
circuit = Circuit(3)
circuit.add(gates.X(i) for i in range(3))
circuit.add(gates.M(i) for i in range(3))
visualize_states(circuit(nshots=1000).frequencies())

#### Let's simulate some entanglement

We can simulate the smallest entangling system in order to reproduce one of the Bell's states

$$ |b_1\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}} \\. $$

To do this, we need to create a two-qubit circuit, lead one of the two qubits to a superposed state using an Hadamard gate and then apply a controlled-NOT gate to the second qubit using the superposed one as control.

<center><img src="figures/bell.png" alt="drawing" width="400"/></center>

In [ ]:
# two qubit circuit to simulate the first Bell's state
c = Circuit(2)
c.add(gates.H(0))
c.add(gates.CNOT(q0=0, q1=1))
c.add(gates.M(*range(2)))

In [ ]:
# collect outcome and frequencies
freq = c(nshots=1000).frequencies(binary=True)

# visualize it
visualize_states(freq)

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
    <strong>Exercise:</strong> implement the quantum circuits needed to prepare the other three Bell's states:
    $$ |b_2\rangle = \frac{|00\rangle - |11\rangle}{\sqrt{2}},\qquad |b_3\rangle = \frac{|10\rangle + |01\rangle}{\sqrt{2}},\qquad |b_4\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}} \\. $$
</div>

In [ ]:
circuit = Circuit(2)
circuit.add(gates.H(0))
circuit.add(gates.Z(0))
circuit.add(gates.CNOT(0,1))
print(circuit())

In [ ]:
circuit = Circuit(2)
circuit.add(gates.H(0))
circuit.add(gates.X(1))
circuit.add(gates.CNOT(0,1))
print(circuit())

In [ ]:
circuit = Circuit(2)
circuit.add(gates.H(0))
circuit.add(gates.X(1))
circuit.add(gates.Z(0))
circuit.add(gates.CNOT(0,1))
print(circuit())

### Parametrized gates

We can use parametric gates to manipulate the quantum state with some more freedom. 

The most commonly used parametric gates are rotations $R_k(\theta) = \exp [ -i \theta \sigma_k ] $, where $\sigma_k$ is one of the components of the Pauli's vector: $\vec{\sigma}=(I, \sigma_x, \sigma_y, \sigma_z)$.

In [ ]:
# a fancier quantum circuit
nqubits = 2
nlayers = 2

c = Circuit(nqubits=nqubits)

for l in range(nlayers):
    for q in range(nqubits):
        # NOTE: the angles are set to zero here!
        c.add(gates.RY(q=q, theta=0))
        c.add(gates.RZ(q=q, theta=0))
    c.add(gates.CNOT(q0=0, q1=nqubits-1))
c.add(gates.M(*range(nqubits)))

c.draw()

All the rotational angles are now set to zero, and the final state is equal to the initial state (which is $|0\rangle^{\otimes N}$ by default). We can play with the angles to modify the final state.

In [ ]:
# execute the circuit and collect frequencies
outcome = c(nshots=1000)
freq = outcome.frequencies()

print(outcome)
# visualize the |0> state
visualize_states(freq)

Let's now set some random parameters to see how the distribution changes.

In [ ]:
np.random.seed(42)
nparams = len(c.get_parameters())
angles = np.random.randn(nparams)

# set the parameters into the circuit
c.set_parameters(angles)
# execute, collect frequencies and visualize the state
outcome = c(nshots=1000)
freq = outcome.frequencies()

visualize_states(freq)

## Controlled gates
Some controlled gates are already available in Qibo. We can write a generalized controlled gate using `controlled_by`.

In [ ]:
nqubits = 3
circuit = Circuit(nqubits)
circuit.add(gates.X(nqubits - 1).controlled_by(*range(nqubits - 1)))
circuit.add(gates.M(*range(nqubits)))
result = circuit(nshots=1000)

In [ ]:
visualize_states(result.frequencies())

Above we see that we measure only the $|000\rangle$ because the first qubit are both in $|0\rangle$.
Let's see what happens if we set both of them to $|1\rangle$ using $X$ gates.

In [ ]:
nqubits = 3
circuit = Circuit(nqubits)
for i in range(nqubits-1):
    circuit.add(gates.X(i))
circuit.add(gates.X(nqubits - 1).controlled_by(*range(nqubits - 1)))
circuit.add(gates.M(*range(nqubits)))
circuit.draw()
result = circuit(nshots=1000)

In [ ]:
visualize_states(result.frequencies())

## Add circuits as subroutines

Using Qibo we also have the possibility to append a smaller circuit with a bigger circuit using the on_qubits function. For circuits with the same number of qubits you can also the + operator.

In [ ]:
small_qubits = 2
superposition = Circuit(small_qubits)
superposition.add(gates.H(i) for i in range(small_qubits))

In [ ]:
large_qubits = 4
circuit=Circuit(large_qubits)
circuit.add(superposition.on_qubits(0,2))
circuit.add(gates.M(i) for i in range(large_qubits))
circuit.draw()

In [ ]:
visualize_states(circuit(nshots=1000).frequencies())

## Running circuits on an emulator
So far, we have only executed circuits on simulation. To get a glimpse of what is like to run on actual hardware where noise plays a big factor we can use an emulator provided by Qibolab. We will now load an emulator that simulates the behavior of a QPU with just one qubit.

In [ ]:
import os
from qibolab import create_platform

os.environ["QIBOLAB_PLATFORMS"] = "../"
platform = create_platform("emulator")
qibo.set_backend("qibolab", platform=platform)

Since we are now dealing with an emulator not all gates can be executed directly. The gates which are executable direclty are known as <em> native gates </em>. In our case the natives gates are the following:

In [ ]:
from qibo.transpiler import NativeGates
NativeGates.default()

We are now going to define a transpiler for a platform containing only 1 qubit.

In [ ]:
from qibo.transpiler import Passes, Unroller
transpiler = Passes(
    connectivity=[], # this is the topology of the chip, since we are using just a
                     # single qubit, there's no connectivity to respect
    passes=[Unroller(NativeGates.default())]
)
# we set the transpiler 
qibo.set_transpiler(transpiler)

In [ ]:
circuit = Circuit(1)
circuit.add(gates.X(0))
circuit.add(gates.M(0))

print("Circuit before transpilation")
circuit.draw()
print("Circuit after transpilation")
transpiler(circuit)[0].draw()

Having set the transpiler above Qibolab will automatically take care of the transpilation when we run a circuit.

In [ ]:
visualize_states(circuit(nshots=1000).frequencies())